In [1]:
# import libraries and dependecies
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.manifold import TSNE

In [2]:
# read in csv files and place into dataframes
# Read in the transaction_fraud_data.csv file into a PandasDataFrame.

fraud_data = pd.read_csv("User0_credit_card_transactions.csv", index_col="User")

# Review the DataFrame
fraud_data.head()

,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
User,,,,,,,,,,,,,,
0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No


In [8]:
fraud_data.dtypes

Card                int64
Year                int64
Month               int64
Day                 int64
Time               object
Amount             object
Use Chip           object
Merchant Name       int64
Merchant City      object
Merchant State     object
Zip               float64
MCC                 int64
Errors?            object
Is Fraud?          object
dtype: object

In [9]:
#Convert Time and Amount to float
import time

fraud_data['Time']=fraud_data['Time'].time()

fraud_data.dtypes

AttributeError: 'Series' object has no attribute 'time'

In [ ]:
fraud_data['Is Fraud?'].value_counts()

In [ ]:
# The target column should be the binary `Is Fraud` column.
target = fraud_data["Is Fraud?"]

# The features column should be all of the features. 
features = fraud_data.drop(columns="Is Fraud?")

In [ ]:
# Split the dataset using the train_test_split function
training_features, testing_features, training_targets, testing_targets = train_test_split(features, target)

In [ ]:
# Declare a logistic regression model.

logistic_regression_model = LogisticRegression(random_state=7)

In [ ]:
# Fit and save the logistic regression model using the training data
lr_model = logistic_regression_model.fit(training_features, training_targets)

In [ ]:
# Make and save testing predictions with the saved logistic regression model using the test data
testing_predictions = lr_model.predict(testing_features)

# Review the predictions
testing_predictions

In [ ]:
# Display the accuracy score for the test dataset.
accuracy_score(testing_targets, testing_predictions)